# [Подборка индикаторов и их недостатков](https://forexlab-ru.turbopages.org/s/forexlab.ru/indicator-on-balance-volume/)

In [125]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
# Show charts when running kernel
init_notebook_mode(connected=True)
PINK = 'rgba(231,107,243,0.1)'

df = pd.read_csv('spy.us.txt')
df['Close_diff']=df['Close'].diff()
df.head()

,Date,Open,High,Low,Close,Volume,OpenInt,Close_diff
0,2005-02-25,104.77,106.00,104.68,105.79,70221808,0,NaN
1,2005-02-28,105.55,105.68,104.56,105.08,79695344,0,-0.71
2,2005-03-01,105.22,105.87,105.22,105.62,54607412,0,0.54
3,2005-03-02,105.21,106.22,105.10,105.57,73733090,0,-0.05
4,2005-03-03,105.99,106.20,105.15,105.61,71286823,0,0.04


In [126]:
def plot_oscillator(data, oscillator:str):
    """ Рисует график и под ним индикатор"""
    fig = go.Figure(),
    plot1 = go.Scatter(x=data.Date, y=data.Close, yaxis="y2", name='Close', line_color='dimgray', opacity=0.3)
    plot2 = go.Scatter(x=data.Date, y=data.loc[:,oscillator], yaxis="y1",  name=oscillator, fill='tonexty', fillcolor=PINK)
    
    data = [plot1, plot2]
    layout = go.Layout(yaxis1=dict(domain=[0, 0.33]), yaxis2=dict(domain=[0.33, 1]))
    fig = go.Figure(data=data, layout=layout)
    fig.show()

# 1. RSI
# $ \frac{Avg(PriceUp)}{Avg(PriceUP)+Avg(PriceDown)}*100 $
###  PriceUp(t) = $ Price(t)-Price(t-1)$ `price > 0` 

###  PriceDown(t) = $ abs((Price(t)-Price(t-1)))$`price < 0`


In [127]:
def RSI(df, n=14):
    delta = df['Close_diff']
    pricesUp   = delta.copy()
    pricesDown = delta.copy()
    
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    
    rollUp   = pricesUp.rolling(n).mean()
    rollDown = -1*pricesDown.rolling(n).mean()
    rsi = 100 * rollUp / (rollUp + rollDown)
    return rsi

In [128]:
df['RSI_14D'] = RSI(df).fillna(0)

In [129]:
plot_oscillator( df[-200:], 'RSI_14D')

# 2. Bollinger Bands
1. Средняя линия ***Avg*** *(обычное скользящее среднее)* 
2. Верхняя линия ***TL*** (средняя линия ***Avg***, смещенная вверх на определенное число ***D*** стандартных отклонений Std)
3. Нижняя линия ***BL*** (средняя линия ***Avg***, смещенная вниз на число стандартных ***D*** отклонений Std)
### $TL|BL = Avg_{30} ± (D * Std_{30})$

[Std](https://youtu.be/GqMOty05n44) – стандартное отклонение рассчитывается как:

   ### $Std = \sqrt{\frac{\sum(Price_i — Avg_{30})^{2}}  {N}}$


In [130]:
def bbands(df, length=30, numsd=2):
    close = df['Close']
    ave = close.rolling(length).mean()
    std  = close.rolling(length).std()
    upband = ave + (std * numsd)
    dnband = ave - (std * numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

In [131]:
df = df.assign(**dict(zip(
                ('BB_Middle_Band', 'BB_Upper_Band', 'BB_Lower_Band'),
                bbands(df))))

In [132]:
fig, data = go.Figure(), df[-200:]
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray', opacity=0.3))

fig.add_trace(go.Scatter(x=data.Date, y=data.BB_Upper_Band,  name='BB_ Upper_Band'))
fig.add_trace(go.Scatter(x=data.Date, y=data.BB_Lower_Band,  name='BB_Lower_Band',fill='tonexty',fillcolor=PINK))
fig.add_trace(go.Scatter(x=data.Date, y=data.BB_Middle_Band, name='BB_Middle_Band',))

# 3. Aroon Oscillator [AO](https://equity.today/indikator-aroon.html)  
### $ AO = AroonUp − AroonDown $
### $ AroonUp|Down = 100*\frac{25-PeriodsSince(High|Low)_{25}}{25} ,\;25 =\;количеству\: периодов$

In [133]:
# idx_local_high= количеству дней от начала до локального макс/мин
# если макс/мин сегодня, то переменная равна N
# если макс/мин первый день, то переменная равна 1 

def aroon(df, N=25):
    aroonup = []
    aroondown = []
    x = N
    while x<= len(df['Date']):
        slice_h =list(df['High'][x-N:x])
        slice_l =list(df['Low'][x-N:x])
        
        idx_local_high = 1 + slice_h.index(max(slice_h))
        idx_local_low =  1 + slice_l.index(min(slice_l))
        
        aroonup.append(idx_local_high/N*100)
        aroondown.append(idx_local_low/N*100)
        x+=1
    return aroonup, aroondown

In [134]:
listofna = [None] * 24
up, down = aroon(df)
aroon_list = [x - y for x, y in zip(up,down)]
df['Aroon_25D'] = listofna + aroon_list

In [135]:
plot_oscillator( df[-200:], 'Aroon_25D')

# 4. Price Volume Trend [PVT](https://www.metastock.com/customer/resources/taaz/default.aspx?p=93)
### $PVT = [\frac{CurrentClose - PreviousClose} {PreviousClose} * Volume] + PreviousPVT $

In [136]:
def pvt(df):
    df['PVT']=None
    PreviousPVT=0
    for stock in range(1,len(df)):
        PVT =   df.loc[stock, 'Close_diff'] / df.loc[stock-1, 'Close'] * df.loc[stock, 'Volume']
        df.loc[stock, 'PVT'] = PVT + PreviousPVT
        PreviousPVT = PVT

In [137]:
pvt(df)

In [138]:
plot_oscillator( df[-200:], 'PVT')

# 5. Acceleration Bands [AB](https://fx-roboti.ru/acceleration-bands.html)
### $Upper Band =SMA(High* ( 1 ±^{?} 4 *\frac{High - Low}{High + Low})) $
### $Middle Band = SMA_{}$
### $Lower Band = SMA(Low *
 (1 - 4 * \frac{High - Low}{High + Low}))$
- SMA =  Simple Moving Average

In [139]:
def abands(df):
    df['AB_Middle_Band'] = df['Close'].rolling(20).mean()
    ab_up  = df['High'] * (1 + 4 * (df['High']-df['Low'])/(df['High']+df['Low']))
    ab_down = df['Low'] * (1 - 4 * (df['High']-df['Low'])/(df['High']+df['Low']))

    df['AB_Upper_Band'] = ab_up.rolling(20).mean()
    df['AB_Lower_Band'] = ab_down.rolling(20).mean()

In [140]:
abands(df)

In [141]:
fig, data = go.Figure(), df[-200:]
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray', opacity=0.3))

fig.add_trace(go.Scatter(x=data.Date, y=data.AB_Upper_Band,  name='AB_ Upper_Band'))
fig.add_trace(go.Scatter(x=data.Date, y=data.AB_Lower_Band,  name='AB_Lower_Band',fill='tonexty',fillcolor=PINK))
fig.add_trace(go.Scatter(x=data.Date, y=data.AB_Middle_Band, name='AB_Middle_Band',))

# 6. Stochastic Oscillator (%K and %D)
тут я вместо $L_{min}$ использовал Avg(L)<br>
__смотри 1 часть__

In [142]:
def stochastic(df, k=14, d=3):
    df['stoch_k'] = ((df['Close'] - df['Low'].rolling(k).mean()) / (df['High'].rolling(k).max() - df['Low'].rolling(k).min())) * 100
    df['stoch_d'] = df['stoch_k'].rolling(d).mean()    

In [143]:
stochastic(df)

In [144]:
plot_oscillator( df[-200:], 'stoch_d')

In [145]:
fig, data = go.Figure(), df[-190:]
fig.add_trace(go.Scatter(x=data.Date, y=data.stoch_d,  name='stoch_d'))

# 7. Chaikin Money Flow [CMF](https://www.tradingview.com/wiki/Chaikin_Money_Flow_(CMF)/ru)

### - Найдите множитель денежных потоков<br>  $ \frac{(Close  -  Low) - (High - Close)}{(High - Low)} =  Money Flow Multiplier$ 

### - Рассчитать объем денежных потоков
*Money Flow Multiplier x Volume for the Period* (Объем за период) = Money Flow Volume

### - Вычислить CMF
Сумма значений  MoneyFlowVolume за весь период / сумме объемов за тот же период

`Идеома`: 1. пересечение индикатором нулевой отметки 2. дивергенция

In [146]:
def CMFolow(df, n):
    PeriodVolume = df['Volume'].rolling(n).sum()
    
    MF_volume = (2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low']) * PeriodVolume
    Period_MF_volume = MF_volume.rolling(n).sum()
    
    CMF = Period_MF_volume/PeriodVolume
    return CMF

df['Chaikin_MF'] = CMFolow(df, 20)

In [147]:
plot_oscillator( df[-200:], 'Chaikin_MF')

# 8. Parabolic [SAR](https://smart-lab.ru/finansoviy-slovar/Индикатор%20Parabolic%20SAR)
Uptrend Parabolic SAR = AF*(HIGHT – PreviousSAR) + PreviousSAR 

Downtrend Parabolic SAR = AF*(LOW – PreviousSAR) – PreviousSAR

***SAR*** –  текущее значение ордера на закрытие позиции<br>
***SAR'*** – новое значение ордера на закрытие позиции<br>
***AF*** –  фактор ускорения, шаг изменения цены закрытия позиции, рекомендуемое значение 0,02   


### $AF = 0.2 + i * 0.02$
i - число периодов после начала расчета <br>
0.02 – шаг изменения цены<br>
0.2 – максимум <br> 

In [148]:
def psar(df, iaf = 0.02, maxaf = 0.2):
    """
    return: psar for all, [psar for bear trend, psar for bull trend]
    """
    length = len(df)
    high = df['High']
    low  = df['Low']
    close= df['Close']
    psar = close.copy()
    
    psarbull = [None] * length
    psarbear = [None] * length
    bull = True
    af = iaf
    local_high = high[0]
    local_low = low[0]
    for i in range(2,length):
        # _____________________ определяем формулу и считаем sar'
        if bull:
            psar[i] = psar[i - 1] + af * (local_high - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (local_low - psar[i - 1])
        
        # _____________________ если разворот, то обновить переменные
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = local_high
                local_low = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = local_low
                local_high = high[i]
                af = iaf
                
        if not reverse:
            if bull:
                if high[i] > local_high:
                    local_high = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < local_low:
                    local_low = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
        if bull:
            psarbull[i] = psar[i]
        else:
            psarbear[i] = psar[i]
    return psar, [psarbear, psarbull]

In [149]:
df['Parabolic_SAR'] = psar(df)[0]

In [150]:
data = df[-200:]
fig = px.scatter(x=data.Date, y=data.Parabolic_SAR)
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray'))

# 9. Price Rate of Change [ROC](https://www.mql5.com/ru/code/46)
 Осциллятор скорости Изменения Цены <br>Чем больше ценовое изменение, тем сильнее меняется ROC
## $ROC = \frac{CLOSE_i - CLOSE_{i - 25}}{CLOSE_{i - 25}}* 100$

In [151]:
def rate_of_change(df, n):
    M = df['Close'].diff(n)
    N = df['Close'].shift(n)
    ROC = M / N * 100
    return ROC

In [152]:
df['%ROC'] = rate_of_change(df, 25)

In [153]:
plot_oscillator( df[-200:], '%ROC')

# 10. Volume Weighted Average Price [VWAP](https://smart-lab.ru/blog/415940.php)
используется для измерения средней цены, взвешенной по объёму
1. Вычислите Типичную цену за период.$ [(High + Low + Close)/3)]$
2. Умножьте Типичную цену на период Объем $(Typical Price * Volume)$
3. Создайте совокупную *(кумулятивную)* сумму типичной цены.  $Cumulative(Typical Price x Volume)$
4. Создайте совокупное общее количество. Cumulative(Volume)
5. Разделите кумулятивные итоги. 

## $VWAP = \frac{Cumulative(Typical Price * Volume)}{ Cumulative(Volume)}$

In [154]:
df['VWAP'] = np.cumsum(df['Volume'] * (df['High'] + df['Low'])/2) / np.cumsum(df['Volume'])

In [155]:
fig, data = go.Figure(), df[:-900]
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray'))
fig.add_trace(go.Scatter(x=data.Date, y=data.VWAP, name='VWAP'))

# 11. [Momentum](https://bcs-express.ru/novosti-i-analitika/indikator-skorosti-rynka-momentum)
## $MOMENTUM = \frac{CLOSE_i}{CLOSE_{i - n}} * 100$


In [156]:
def momentum(df, n):
    return (df['Close']/df['Close'].shift(n))* 100
df['%Momentum']=momentum(df, 14)

# 12. Commodity Channel Index [CCI](http://tlap.com/indikator-commody-channel-index/) 
### $CCI = \frac{Typical Price – SMA}{0,015*Avg}$
Предполагается, что если цена отклоняется от своей скользящей средней на интервал, больший чем обычный для рассматриваемого периода, то происходит изменение тренда

In [157]:
def CCI(df, n, constant):
    TP = (df['High'] + df['Low'] + df['Close']) / 3
    CCI = (TP - TP.rolling(n).mean()) / (constant * TP.rolling(n).std())
    return CCI

In [158]:
df['CCI'] = CCI(df, 20, 0.015)

In [159]:
plot_oscillator( df[100:300], 'CCI')

# 13. On Balance Volume [OBV](http://tlap.com/on-balance-volume/)
Суммирует или вычитает объём<br>
$
\textit{OBV}_t = \textit{OBV}_{t-i} + \left\{ \begin{matrix}
\textit{volume}_t  & \mathrm{if}\ \textit{close}_t > \textit{close}_{t-i} \\
0       & \mathrm{if}\ \textit{close}_t = \textit{close}_{t-i} \\
-\textit{volume}_t & \mathrm{if}\ \textit{close}_t < \textit{close}_{t-i}
\end{matrix} \right.
$


In [160]:
diff = df['Close'].diff()
obv = (df['Volume'] * (np.where(diff< 0, -1, 1)*np.where(diff== 0, 0, 1))).cumsum()

df['OBV'] = obv

In [191]:
data = df[-200:]
go.Figure(go.Bar(x=df.Date, y=df.OBV, name='OBV', marker_color='red'))

# 14. Keltner Channels [KC](https://smart-lab.ru/blog/239825.php)
***Канал Кёльтнера*** — индикатор, состоящий из двух полос, построенных вокруг SMA, ширина которых зависит от волатильности рынка.
- Тут стандартнвя реализация, без [модификаций](https://ru.wikipedia.org/wiki/Канал_Кельтнера)

$ UpLine = SMA_{10}(TypicalPrice) + SMA_{10}(TradingRange))$<br>
$ DownLine = SMA_{10}(TypicalPrice)- SMA_{10}(TradingRange))$

$TradingRange = High - Low$<br>
$TypicalPrice = (High + Low + Close )/3$

Считается, что пересечение верхней линии является сигналом к открытию длинной позиции и нижней — короткой

In [162]:
def KELCH(df, n):  
    TypicalPrice = (df['High'] + df['Low'] + df['Close']) / 3
    TradingRange =  df['High'] - df['Low']
    
    KelChM = TypicalPrice.rolling(n).mean()
    KelChU = (TypicalPrice + TradingRange).rolling(n).mean()
    KelChD = (TypicalPrice - TradingRange).rolling(n).mean()    
    return KelChD, KelChM,  KelChU

In [163]:
df = df.assign(**dict(zip(
                ('Kelch_Down', 'Kelch_Middle', 'Kelch_Upper'),
                KELCH(df, 10))))

In [164]:
fig, data = go.Figure(), df[-200:]
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray', opacity=0.5))

fig.add_trace(go.Scatter(x=data.Date, y=data.Kelch_Down,  name='Kelch_Down'))
fig.add_trace(go.Scatter(x=data.Date, y=data.Kelch_Upper, name='Kelch_Upper',fill='tonexty',fillcolor=PINK))
fig.add_trace(go.Scatter(x=data.Date, y=data.Kelch_Middle,name='Kelch_Middle'))

# 15. Triple Exponential Moving Average [TRIX](https://fx-wiki.ru/wiki/Triple_Exponential_Moving_Average(TRIX))
является осциллятором перекупленности и перепроданности. Отличается от МА тем, что выдаёт меньше ложных сигналов.
###  Различные реализации 
$TEMA1 =(3∗EMA_1)−(3∗EMA_1*EMA_1)+3∗EMA_1*EMA_1*EMA_1$<br>
$TEMA2 =(3∗EMA_1)−(3∗EMA_2)+EMA_3$<br>
$TEMA3 =\frac{(EMA_3(i))−(EMA_3(i-1))}{EMA_3(i-1)}$


$EMA_1 = EMA$<br>
$EMA_2 = EMAofEMA_1$<br>
$EMA_3=EMAofEMA_2$<br>


In [165]:
def triple_exponential_moving_average_1(df, n=14):
    EMA =df['Close'].ewm(span=n).mean()
    return (3 * EMA) - (3 * EMA * EMA) + (EMA * EMA * EMA)

def triple_exponential_moving_average_2(df, n):
    EX1 = df['Close'].ewm(span=n).mean()
    EX2 = EX1.ewm(span=n).mean()
    EX3 = EX2.ewm(span=n).mean()
    return (3 * EX1)  - (3 * EX2) + EX3
     
            
def triple_exponential_moving_average_3(df, n):
    EX1 = df['Close'].ewm(span=n).mean()
    EX2 = EX1.ewm(span=n).mean()
    EX3 = EX2.ewm(span=n).mean()
    return (EX3 - EX3.shift(1) )/ EX3.shift(1)

# 16. Normalized Average True Range [ATR](http://tlap.com/indikator-atr/)

Средний Истинный Диапазон –  индикатор, отражающий волатильность движения актива.<br>
Основной смысл индикатора ATR заключается в определении среднего диапазона изменения цены за определенный период времени.
                        
###  $True Range = max(High_i - Low_i, abs(High_i - Close_{i-1}), abs(Low_i - Close_{i-1}))$

$ATR = SMA_{14}(TrueRange)$

Normalized Average True Range = ATR / Close * 100

In [166]:
def ATR(df, n):
    """:return Normalized ATR, ATR """
    df = df.copy()
    df['tradingRange']= df['High'] - df['Low']
    df['absHC'] = abs(df['High'] - df['Close'].shift(1))
    df['absLC'] = abs(df['Low'] - df['Close'].shift(1))

    trueRange = df[['tradingRange','absHC','absLC']].max(axis=1)
    df['ATR'] = trueRange.rolling(14).mean()
    return (df['ATR'] / df['Close']) *100,  df['ATR']

In [167]:
df['%ATR'],df['ATR'] = ATR(df, 14)

In [168]:
plot_oscillator( df[300: 1500], '%ATR')

# 17. Average Directional Movement Index [ADX](https://smart-lab.ru/finansoviy-slovar/индикатор%20ADX)
среднее направленние движения — индикатор для измерения силы тренда
хорошо зарекомендовал себя в целях поиска бумаг, которые пребывают в тренде, либо консолидируются
<br>подтверждает сигналы, даваемые другими индикаторами.[SAR+ADX](https://www.azbukatreydera.ru/sekrety-indikatora-adx.html)
### *1. Directional Movement*<br> $ UpMove = High_i - High_{i-1}$<br>$ DownMove = Low_{i-1} - Low_i$

$if\ UpMove > DownMove\ \&\ UpMove > 0:\ +DM = UpMove$ <br> **else**: $+DM = 0$<br> 
$if\ DownMove > UpMove\ \&\ DownMove > 0:\  -DM = DownMove$ <br> **else**: $-DM = 0$


### $High_i -  High_{i-1} = UpMove$ <br>$ Low_{i-1} - Low_i = DownMove$


### *2. Directional Indicator*<br>$+DI = 100*EMA(\frac{+DM}{ Average True Range}) $<br> $-DI = 100*EMA(\frac{-DM}{Average True Range})$

### $\%ADX = 100*EMA(abs(\frac{+DI - -DI}{+DI + -DI}))$

In [169]:
def adx(df, n):
    df = df.copy()
    df['UpMove'] = df['High'] - df['High'].shift(1)
    df['DownMove'] = df['Low'].shift(1) - df['Low']
    df['Zero'] = 0

    df['PlusDM'] = np.where((df['UpMove'] > df['DownMove']) & (df['UpMove'] > df['Zero']), df['UpMove'], 0)
    df['MinusDM'] = np.where((df['UpMove'] < df['DownMove']) & (df['DownMove'] > df['Zero']), df['DownMove'], 0)

    df['plusDI'] = 100 * (df['PlusDM']/df['ATR']).ewm(span=n).mean()
    df['minusDI'] = 100 * (df['MinusDM']/df['ATR']).ewm(span=n).mean()

    return 100 * (abs((df['plusDI'] - df['minusDI'])/(df['plusDI'] + df['minusDI']))).ewm(span=n).mean()

In [170]:
df['%ADX'] = adx(df, 14)

In [171]:
plot_oscillator( df[-200:], '%ADX')

# 18. Money Flow Index [MFI](https://forexlab-ru.turbopages.org/s/forexlab.ru/money-flow-index-mfi-description/)
 призванный показать интенсивность, с которой деньги вкладываются в ценную бумагу и выводятся из неё, анализируя объёмы торгов и соотношения типичных цен периодов

$Typical Price = (High + Low + Close)/3$

$Raw Money Flow = Typical Price * Volume$

Денежный поток делится на положительный и отрицательный

### Positive money flow = $typicalPrice_i > typicalPrice_{i-1} $ => typicalPrice x Volume<br>Negative money flow = $typicalPrice_i < typicalPrice_{i-1} $=> typicalPrice x Volume

Money Flow Ratio = (14-period Positive Money Flow)/(14-period Negative Money Flow)

Money Flow Index = 100 - 100/(1 + Money Flow Ratio)

In [172]:
def MFI(df):
    df = df.copy()
    typicalPrice = (df['High']+df['Low']+df['Close'])/3
    rawMoneyFlow = typicalPrice * df['Volume']
    df['Positive MF'] = np.where(typicalPrice > typicalPrice.shift(1), rawMoneyFlow, 0)
    df['Negative MF'] = np.where(typicalPrice < typicalPrice.shift(1), rawMoneyFlow, 0)
    ratio_mf = df['Positive MF'].rolling(window=14).sum()/df['Negative MF'].rolling(14).sum()
    return  100 - 100 / (1 + ratio_mf)

In [173]:
df['%MFI'] = MFI(df)

In [174]:
plot_oscillator( df[-900:], '%MFI')

# 19. Ichimoku Cloud
[Ichimoku kinkou-hyou](https://ru.wikipedia.org/wiki/Индикатор_Ишимоку) — «таблица равновесия цен, которую можно охватить одним взглядом»


***Turning Line*** = ( Highest High + Lowest Low ) / 2, for the past `9 days`

***Standard Line*** = ( Highest High + Lowest Low ) / 2, for the past `26 days`

***Leading Span 1*** = ( ***Standard Line + Turning Line*** ) / 2, plotted `26 days` ahead of today

***Leading Span 2*** = ( Highest High + Lowest Low ) / 2, for the past `52 days`, plotted `26 days` ahead of today

***Cloud*** = Shaded Area between ***Span 1*** and ***Span 2***

***«Мертвый крест»*** — пересечение сверху вниз тенкан и киджун. Сигнал для открытия короткой позиции.<br>
***«Золотой крест»*** — тенкан пересекает киджун снизу вверх. Сигнал для открытия длинной позиции.<br>
Если цена находится в «облаке» — это боковое движение рынка.


In [175]:
def ichimoku(df):
    # Turning Line
    period9_high = df['High'].rolling(9).max()
    period9_low  = df['Low'].rolling(9).min()
    df['turning_line'] = (period9_high + period9_low) / 2
    
    # Standard Line
    period26_high = df['High'].rolling(26).max()
    period26_low  = df['Low'].rolling(26).min()
    df['standard_line'] = (period26_high + period26_low) / 2
    
    # Leading Span 1
    df['ichimoku_span1'] = ((df['turning_line'] + df['standard_line']) / 2).shift(26)
    
    # Leading Span 2
    period52_high = df['High'].rolling(52).max()
    period52_low  = df['Low'].rolling(52).min()
    df['ichimoku_span2'] = ((period52_high + period52_low) / 2).shift(26)
    
    # The most current closing price plotted 22 time periods behind
    df['chikou_span'] = df['Close'].shift(-22)

In [198]:
fig, data = go.Figure(), df[-200:]
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray'))

fig.add_trace(go.Scatter(x=data.Date, y=data.turning_line, name='Tenkan-sen',line_color='red'))
fig.add_trace(go.Scatter(x=data.Date, y=data.standard_line,name='Kijun-sen', line_color='cyan'))

fig.add_trace(go.Scatter(x=data.Date, y=data.ichimoku_span1,name='ichimoku_span1', line_color='pink'))
fig.add_trace(go.Scatter(x=data.Date, y=data.ichimoku_span2,name='ichimoku_span2',fill='tonexty',line_color='pink', fillcolor=PINK))

fig.add_trace(go.Scatter(x=data.Date, y=data.chikou_span, name='Chikou-span', line_color='green'))

# 20. Williams percent range [%WR](https://bcs-express.ru/novosti-i-analitika/volshebnyi-indikator-williams-r)
определяет состояние перекупленности/перепроданности по положению текущей цены закрытия в диапазоне между минимумом и максимумом цен за предыдущие периоды
### $ \%R = -100 * \frac{ Highest High - Close}{ Highest High - Lowest Low } $


In [199]:
def WillR(df):
    highest_high = df['High'].rolling(14).max()
    lowest_low = df['Low'].rolling(14).min()
    return (-100) * ((highest_high - df['Close']) / (highest_high - lowest_low))

In [200]:
df['%WillR'] = WillR(df)

In [202]:
plot_oscillator( df[-900:], '%WillR')

# 21. MINMAX
Индикатор служит для определения сильных уровней поддержки и сопротивления, пробой которых, чаще всего приводит к возникновению тренда.

In [203]:
def MINMAX(df):
    df['MIN_Volume'] = df['Volume'].rolling(14).min()
    df['MAX_Volume'] = df['Volume'].rolling(14).max()

# 21. Adaptive Moving Average [KAMA](http://o-s-a.net/articles/kama.html)
модифицированный MA имеющий минимальную чувствительность к шумовым колебаниям рынка.<br>
Период этой МА постоянно адаптируется к текущей волатильности рынка, удлиняясь или укорачиваясь в зависимости от амплитуды колебаний.
[формулы](https://ru.wikipedia.org/wiki/Адаптивная_скользящая_средняя_Кауфмана)

***efficiency ratio*** - *коэффициент эффективности* основан на соотношении общего движения цены *(direction)* и суммы абсолютных значений шумовых движений рынка *(volatility)* за определенный период(n)

***scaled smoothing constant*** - изменяющаяся сглаживающая константа,<br> строится исходя из предположения, что в завимиости от *коэффициента эффективности* она должна «помнить» данные за разное количество предыдущих периодов. <br>То есть на трендовом рынке следует применять быструю скользящую среднюю, а на нетрендовом — медленную

### ${\textit {sc}}=({\textit {EfficiencyRatio}}\cdot ({\textit {fastest}}-{\textit {slowest}})+{\textit {slowest}})^{2}$
### $KAMA[i] = sc * Close[i]+(1-sc) * KAMA[i-1]$

In [181]:
def KAMA(close, n=10, f=2, s=30):
    direction  = abs( close - close.shift(n))
    absDiffx = abs(close - close.shift(1))  
    volatility = absDiffx.rolling(n).sum()
    ER = direction / volatility

    fastest = 2/(f+1)
    slowest = 2/(s+1)
    sc = (ER*(fastest - slowest) + slowest) ** 2

    answer = np.zeros(len(close))
    for i in range(n,len(close)):
            answer[i] = answer[i-1] + sc[i] * (close[i] - answer[i-1])
    return answer

In [182]:
df['KAMA'] = KAMA(df['Close'])

In [225]:
fig, data = go.Figure(), df[-500:-300]
fig.add_trace(go.Scatter(x=data.Date, y=data.Close, name='Close', line_color='dimgray'))
fig.add_trace(go.Scatter(x=data.Date, y=data.KAMA, name='AMA'))